In [ ]:
!unzip test_Bh8pGW3.zip


In [ ]:
!ls Test

In [ ]:
!unzip train_DETg9GD.zip

In [ ]:
!head train.csv

In [ ]:
!unzip IMFDB_final.zip

In [ ]:
!head IMFDB_final/AamairKhan/3Idiots/3Idiots.txt

In [29]:
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [7]:
'''from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_df['Class'] = le.fit_transform(train_df['Class'])'''


In [9]:
train_df.head(5)

,ID,Class
0,377.jpg,0
1,17814.jpg,2
2,21283.jpg,0
3,16496.jpg,2
4,4487.jpg,0


In [24]:
train['Class'].unique()

array(['MIDDLE', 'YOUNG', 'OLD'], dtype=object)

In [26]:
train.shape

(19906, 2)

In [18]:

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping


img_width, img_height = 64, 64
train_data_dir = "Train"
validation_data_dir = "Train"
nb_train_samples = 0.8*len(train_df)
nb_validation_samples = 0.2*len(train_df) 
batch_size = 16
epochs = 50

def normalCNN():
    
    model = applications.ResNet50(include_top=False, weights='imagenet', input_shape=(img_width, img_height, 3))
    
    # Adding custom Layer
    # We only add
    x = model.output
    x = Flatten()(x)
    # Adding even more custom layers
    x = Dense(1024, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation="relu")(x)
    predictions = Dense(3, activation="softmax")(x)

    # creating the final model 
    model_final = Model(input = model.input, output = predictions)
    
    # Freeze the layers which you don't want to train. Here I am freezing the all layers.
    for layer in model_final.layers[:-5]:
        layer.trainable = False
        
    return model_final


In [19]:
#Custom logic to read labels in-memory from the csv file, and omages as a sequence from image folder
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.utils import Sequence
import math
import random

train_datapath = 'Train/'

def load_image(im):
    img_array = img_to_array(load_img(im, grayscale=True)) / 255.
    return np.reshape(img_array, (img_width, img_height, 3))


In [10]:
'''class ImgDataSequence(Sequence):
    """
    Keras Sequence object to train a model on larger-than-memory data.
    """
    def __init__(self, df, data_path, batch_size, mode='train'):
        self.df = df
        self.bsz = batch_size
        self.mode = mode

        # Take labels and a list of image locations in memory
        self.labels = self.df['Class'].values
        self.im_list = self.df['ID'].apply(lambda x: os.path.join(data_path, x)).tolist()
        
    def __len__(self):
        return int(math.ceil(len(self.df) / float(self.bsz)))

    def on_epoch_end(self):
        # Shuffles indexes after each epoch if in training mode
        self.indexes = range(len(self.im_list))
        if self.mode == 'train':
            self.indexes = random.sample(self.indexes, k=len(self.indexes))

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return self.labels[idx * self.bsz: (idx + 1) * self.bsz]
    
    def get_batch_features(self, idx):
        # Fetch a batch of inputs
        return np.array([load_image(im) for im in self.im_list[idx * self.bsz: (1 + idx) * self.bsz]])

    def __getitem__(self, idx):
        batch_x = self.get_batch_features(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_x, batch_y'''



'class ImgDataSequence(Sequence):\n    """\n    Keras Sequence object to train a model on larger-than-memory data.\n    """\n    def __init__(self, df, data_path, batch_size, mode=\'train\'):\n        self.df = df\n        self.bsz = batch_size\n        self.mode = mode\n\n        # Take labels and a list of image locations in memory\n        self.labels = self.df[\'Class\'].values\n        self.im_list = self.df[\'ID\'].apply(lambda x: os.path.join(data_path, x)).tolist()\n        \n    def __len__(self):\n        return int(math.ceil(len(self.df) / float(self.bsz)))\n\n    def on_epoch_end(self):\n        # Shuffles indexes after each epoch if in training mode\n        self.indexes = range(len(self.im_list))\n        if self.mode == \'train\':\n            self.indexes = random.sample(self.indexes, k=len(self.indexes))\n\n    def get_batch_labels(self, idx):\n        # Fetch a batch of labels\n        return self.labels[idx * self.bsz: (idx + 1) * self.bsz]\n    \n    def get_batch_f

In [20]:
# Initiate the train and test generators with data Augumentation 

datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

train_generator=datagen.flow_from_dataframe(dataframe=train_df, directory="Train/", x_col="ID", y_col="Class", subset="training",
batch_size=32, seed=42, shuffle=True, class_mode="categorical", target_size=(img_width, img_height))

valid_generator=datagen.flow_from_dataframe(dataframe=train_df, directory="Train/", x_col="ID", y_col="Class", subset="validation",
batch_size=32, seed=42, shuffle=True, class_mode="categorical", target_size=(img_width, img_height))

test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(dataframe=test_df, directory="Test/", x_col="ID", y_col=None, batch_size=32,
seed=42, shuffle=False, class_mode=None, target_size=(img_width, img_height))

Found 14930 images belonging to 3 classes.
Found 4976 images belonging to 3 classes.
Found 6636 images.


In [24]:
train_df['Class'].value_counts()

MIDDLE    10804
YOUNG      6706
OLD        2396
Name: Class, dtype: int64

In [27]:
import os
import numpy as np


# compile the model
model_standard_cnn = normalCNN()
model_standard_cnn.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(), metrics=["accuracy"])

# Save the model according to the conditions  
checkpoint = ModelCheckpoint("resnet50_retrain.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

from sklearn.utils import compute_class_weight
classWeight = compute_class_weight('balanced', train_df['Class'].unique(), train_df['Class']) 
classWeight = dict(enumerate(classWeight))

# Train the model 
model_standard_cnn.fit_generator(
  train_generator,
  steps_per_epoch = nb_train_samples//batch_size,
  epochs = 5, #FOR SPEED KEEPING IT TO 5 FOR NOW
  validation_data = valid_generator, #TODO: 1 MAJOR QUERY HERE IS THAT CAN TRAIN AND VALIDATION SAMPLES EVER BE SAME EVEN THOUGH VALIDATION_SPLIT IS APPLIED ??
  validation_steps = nb_validation_samples//batch_size,
  callbacks = [checkpoint, early], workers = 4, class_weight = classWeight)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:34: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


Epoch 1/5
995/995 [==============================] - 505s 508ms/step - loss: 1.1269 - acc: 0.5132 - val_loss: 1.0744 - val_acc: 0.3481

Epoch 00001: val_acc improved from -inf to 0.34811, saving model to resnet50_retrain.h5
Epoch 2/5
995/995 [==============================] - 475s 477ms/step - loss: 0.7966 - acc: 0.6054 - val_loss: 1.0055 - val_acc: 0.5416

Epoch 00002: val_acc improved from 0.34811 to 0.54162, saving model to resnet50_retrain.h5
Epoch 3/5
995/995 [==============================] - 467s 470ms/step - loss: 0.6736 - acc: 0.6771 - val_loss: 1.0335 - val_acc: 0.5333

Epoch 00003: val_acc did not improve from 0.54162
Epoch 4/5
995/995 [==============================] - 467s 469ms/step - loss: 0.5851 - acc: 0.7279 - val_loss: 0.9544 - val_acc: 0.5362

Epoch 00004: val_acc did not improve from 0.54162
Epoch 5/5
995/995 [==============================] - 464s 466ms/step - loss: 0.4970 - acc: 0.7752 - val_loss: 0.9724 - val_acc: 0.5345

Epoch 00005: val_acc did not improve from

In [ ]:
'''
TODO:
1. Remove the last few FC layers from Resnet50, add the custom layers and train. This will reduce this overfitting problem
- Lets try this first - there are no FC layers in Keras Resnet ???

2. Use facenet
3. Use Debiasing Variational Autoencoder
'''

In [26]:
model = applications.ResNet50(include_top=False, weights='imagenet', input_shape=(img_width, img_height, 3))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_10[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [30]:
!git clone https://github.com/iwantooxxoox/Keras-OpenFace

Cloning into 'Keras-OpenFace'...
remote: Enumerating objects: 383, done.
remote: Total 383 (delta 0), reused 0 (delta 0), pack-reused 383
Receiving objects: 100% (383/383), 109.63 MiB | 8.27 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [38]:
!git clone https://github.com/JudasDie/deeplearning.ai

Cloning into 'deeplearning.ai'...
remote: Enumerating objects: 679, done.
remote: Total 679 (delta 0), reused 0 (delta 0), pack-reused 679
Receiving objects: 100% (679/679), 40.56 MiB | 8.15 MiB/s, done.
Resolving deltas: 100% (276/276), done.


In [56]:
!cp deeplearning.ai/"Convolutional Neural Networks"/week4/FaceNet/* .

In [ ]:
from deeplearning.ai

In [59]:
from inception_blocks_v2 import faceRecoModel
#from keras.utils import CustomObjectScope  

def facenetModel():
    FRmodel = faceRecoModel(input_shape=(3, 96, 96))
    return FRmodel
                            
model_facenet = facenetModel()
    

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 128, 12, 192), (None, 32, 12, 192), (None, 32, 12, 102), (None, 64, 12, 192)]